# Next Sentence Prediction

In [1]:
!which python

/home/mostendorff/miniconda2/envs/storytelling-candidates/bin/python


In [2]:
!pip freeze|grep transformers

transformers==4.2.2


In [3]:
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np
import pickle
from smart_open import open
import os
import sys
import random
import json
import pickle
import logging
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from IPython.core.display import display
from collections import defaultdict
import itertools

#sys.path.append(os.path.dirname(os.getcwd()))

from experiments.environment import get_env

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

os.environ['WANDB_DISABLED'] = 'true'

env = get_env()

/data/experiments/mostendorff/storytelling-candidates/environments
Environment detected: gpu_server2 (in default.yml)


In [4]:
!nvidia-smi

Tue Feb 23 15:43:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.31       Driver Version: 440.31       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1B:00.0 Off |                  N/A |
| 27%   32C    P8    11W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 6000     On   | 00000000:1C:00.0 Off |                  Off |
| 33%   26C    P8     4W / 260W |   1511MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  Q

In [5]:
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3,5,6,7'
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,5,6'

In [12]:
from itertools import combinations


from transformers import Trainer, TrainingArguments

from typing import Dict
from transformers.data import metrics
from transformers import EvalPrediction

from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score
from scipy.special import softmax

import torch
from torch.utils.data import Dataset
from transformers import BertForNextSentencePrediction, BertTokenizerFast, BertForSequenceClassification


In [6]:
output_dir = './output/nsp'

data_dir = '/data/experiments/hensel/storytelling-candidates/data'

train_batch_size = 24
eval_batch_size = 28

model_name = 'bert-base-cased'
#model_name = 's2orc-scibert'

model_name_or_path = os.path.join(env['bert_dir'], model_name)

model_output_dir = os.path.join(output_dir, model_name)
model_output_dir

'./output/nsp/bert-base-cased'

### Prepare dataset

In [7]:
meta_df = pd.read_csv(os.path.join(data_dir, 'meta_data-12-02-21.tsv'), sep='\t', index_col=0)
meta_df

,doc_id,start,end,text,url
sent_id,,,,,
736-0,736,0,37,"Hu Jintao, the President of the People's Repub...",https://en.wikinews.org/wiki?curid=736
736-1,736,38,49,Lunch was a traditional Brazilian barbecue wit...,https://en.wikinews.org/wiki?curid=736
741-1,741,0,16,Acting president Rawhi Fattuh has announced to...,https://en.wikinews.org/wiki?curid=741
741-2,741,17,53,"Futtuh, head of the Palestinian parliament, wa...",https://en.wikinews.org/wiki?curid=741
741-3,741,54,84,New leadership could prove to be the key to re...,https://en.wikinews.org/wiki?curid=741
...,...,...,...,...,...
2909884-44,2909884,1032,1042,"Consequently there are water, road and infrast...",https://en.wikinews.org/wiki?curid=2909884
2909884-45,2909884,1043,1065,"We aim to make Groom a marginal electorate, if...",https://en.wikinews.org/wiki?curid=2909884
2909884-46,2909884,1066,1091,My approach is to listen to the community and ...,https://en.wikinews.org/wiki?curid=2909884


In [9]:
train_doc_ids, test_doc_ids = train_test_split(meta_df.doc_id.unique().tolist(), test_size=0.2, random_state=1)

logger.info(f'Train: {len(train_doc_ids)}, Test: {len(test_doc_ids)}')

2021-02-19 11:46:18 - INFO - __main__ -   Train: 13089, Test: 3273


In [9]:
# sub sample
#doc_ids = random.sample(meta_df.doc_id.unique().tolist(), 1000)
#len(doc_ids)

In [17]:

tokenizer = BertTokenizerFast.from_pretrained(model_name_or_path)

In [11]:
# Load model
model = BertForNextSentencePrediction.from_pretrained(model_name_or_path)
#model = BertForSequenceClassification.from_pretrained(model_name_or_path, num_labels=2)

Some weights of the model checkpoint at /home/mostendorff/datasets/BERT_pre_trained_models/pytorch/bert-base-cased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
class NSPDataset(Dataset):
    """   
    0 indicates sequence B is a continuation of sequence A,

    1 indicates sequence B is a random sequence.
    """
    max_length = 256
    neg_ratio = 1.0
    positive_label = 0
    negative_label = 1
    
    def __init__(self, df, doc_ids, hf_tokenizer, return_labels=True, sample_n=0):
        self.df = df
        self.doc_ids = doc_ids
        self.tokenizer = hf_tokenizer
        self.return_labels = return_labels
        self.sample_n = sample_n
        self.inputs = []
        self.samples = []

    def load(self):
        logger.info(f'Dataframe size: {len(self.df)}')
        
        # sub-sample
        if self.sample_n > 0:
            logger.info('Sub-sampling..')
            self.doc_ids = random.sample(self.doc_ids, self.sample_n)
        
        # filter by doc ids
        self.df = self.df[self.df.doc_id.isin(self.doc_ids)]

        # positives
        positive_pairs = set()

        for doc_id, doc_df in self.df.groupby('doc_id'):
            # positive samples
            prev_sentence = None

            for sent in doc_df.sort_values('start')['text']:
                if len(sent) > 50:
                    if prev_sentence is not None:
                        pair = (prev_sentence, sent)

                        positive_pairs.add(pair)

                    prev_sentence = sent
            
        logger.info(f'Postive samples: {len(positive_pairs)}')
        
        # negatives
        negative_pairs = set()        
        neg_needed = int(self.neg_ratio * len(positive_pairs))
        tries = 0

        logger.info(f'Randomly selecting {neg_needed} negative samples (ratio={self.neg_ratio})')
        
        sents = meta_df[['doc_id', 'text']].values.tolist()
        random.shuffle(sents)
        
        rand_sents = sents
        rand_pairs = iter([rand_sents[i:i+2] for i in range(0, len(rand_sents), 2)])

        while len(negative_pairs) < neg_needed:
            (a_doc_id, a_sent), (b_doc_id, b_sent) = next(rand_pairs)
            
            #print(a_doc_id)

            pair = (a_sent, b_sent)
            
            # TODO # and pair not in positive_pairs and pair not in negative_pairs
            if a_doc_id != b_doc_id:
                negative_pairs.add(pair)
            else:
                tries += 1

        logger.info(f'done after {tries} invalid tries')

        logger.info(f'Tokenize... {len(positive_pairs):,} + {len(negative_pairs):,} samples')

        self.inputs = self.tokenizer(
            text=[a for a, b in positive_pairs] + [a for a, b in negative_pairs],
            text_pair=[b for a, b in positive_pairs] + [b for a, b in negative_pairs],
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors='pt',
            padding='max_length',
            max_length=self.max_length,
            truncation=True,
            return_token_type_ids=True
        )
        
        if self.return_labels:
            labels = [self.positive_label] * len(positive_pairs)
            labels += [self.negative_label] * len(negative_pairs)
            self.inputs['labels'] = torch.tensor(labels)

        logger.info('Dataset loaded')

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.inputs.items()}

    def __len__(self):
        return len(self.inputs['input_ids'])

In [15]:
train_ds = NSPDataset(meta_df, train_doc_ids, tokenizer, return_labels=True, sample_n=5000)
train_ds.load()

2021-02-19 11:47:23 - INFO - __main__ -   Dataframe size: 175720
2021-02-19 11:47:23 - INFO - __main__ -   Sub-sampling..
2021-02-19 11:47:24 - INFO - __main__ -   Postive samples: 48189
2021-02-19 11:47:24 - INFO - __main__ -   Randomly selecting 48189 negative samples (ratio=1.0)
2021-02-19 11:47:25 - INFO - __main__ -   done after 4 invalid tries
2021-02-19 11:47:25 - INFO - __main__ -   Tokenize... 48,189 + 48,189 samples
2021-02-19 11:47:33 - INFO - __main__ -   Dataset loaded


In [16]:
test_ds = NSPDataset(meta_df, test_doc_ids, tokenizer, return_labels=True, sample_n=500)
test_ds.load()

2021-02-19 11:47:33 - INFO - __main__ -   Dataframe size: 175720
2021-02-19 11:47:33 - INFO - __main__ -   Sub-sampling..
2021-02-19 11:47:33 - INFO - __main__ -   Postive samples: 4823
2021-02-19 11:47:33 - INFO - __main__ -   Randomly selecting 4823 negative samples (ratio=1.0)
2021-02-19 11:47:34 - INFO - __main__ -   done after 1 invalid tries
2021-02-19 11:47:34 - INFO - __main__ -   Tokenize... 4,823 + 4,823 samples
2021-02-19 11:47:35 - INFO - __main__ -   Dataset loaded


### Training

In [9]:

def compute_metrics(predict_out: EvalPrediction) -> Dict:
    y_true = predict_out.label_ids
    y_pred = np.argmax(predict_out.predictions, axis=1)

    return {
        "f1_micro": f1_score(y_true, y_pred, average='micro'),
        "f1_macro": f1_score(y_true, y_pred, average='macro'),  
        "classification_report": classification_report(
            y_true, y_pred, 
            labels=[0,1], 
            target_names=['pos','neg'],
            output_dict=True,
        ),
        "classification_report_str": classification_report(
            y_true, y_pred, 
            labels=[0,1], 
            target_names=['pos','neg'],
        )
    }


In [19]:

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir=model_output_dir,
        overwrite_output_dir=True,
        learning_rate=2e-5,
        do_train=True,
        num_train_epochs=3,
        per_device_train_batch_size=train_batch_size,  
        per_device_eval_batch_size=eval_batch_size,          
        save_steps=0,
        save_total_limit=3,
        eval_steps=50,
        do_eval=True,
    ),
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [20]:
train_out = trainer.train()
train_out

/home/mostendorff/miniconda2/envs/storytelling-candidates/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.212200
1000,0.186900
1500,0.099700
2000,0.101500
2500,0.048400
3000,0.048700


TrainOutput(global_step=3012, training_loss=0.11593963657994352, metrics={'train_runtime': 1740.6394, 'train_samples_per_second': 1.73, 'total_flos': 48102338876221440, 'epoch': 3.0})

In [21]:
trainer.save_model()
tokenizer.save_pretrained(model_output_dir)

('./output/nsp/bert-base-cased/tokenizer_config.json',
 './output/nsp/bert-base-cased/special_tokens_map.json',
 './output/nsp/bert-base-cased/vocab.txt',
 './output/nsp/bert-base-cased/added_tokens.json')

In [22]:
predict_out = trainer.predict(test_ds)

In [23]:
print(compute_metrics(predict_out)['classification_report_str'])

              precision    recall  f1-score   support

         pos       0.94      0.91      0.93      4823
         neg       0.91      0.94      0.93      4823

    accuracy                           0.93      9646
   macro avg       0.93      0.93      0.93      9646
weighted avg       0.93      0.93      0.93      9646



### Predict for sentences with high similarity

In [13]:

# Create your own dataset
class PredictNSPDataset(Dataset):
    max_length = 256
    
    def __init__(self, sent_id2text, sim_df, hf_tokenizer, min_sim=0., max_sim=1., sample_n=0):
        self.sent_id2text = sent_id2text
        self.sim_df = sim_df
        self.tokenizer = hf_tokenizer
        self.min_sim = min_sim
        self.max_sim = max_sim
        self.sample_n = sample_n
        self.inputs = []
        self.samples = []

    def load(self):

        self.sent1_ids = []
        self.sent2_ids = []

        texts1 = []
        texts2 = []


        logger.info(f'sent_id2text size: {len(self.sent_id2text)}')
        
        if self.sim_df is None:
            # Random pairs
            sent_ids = list(self.sent_id2text.keys())
            sent_id_pairs = combinations(sent_ids, 2)
            
            logger.info(f'Possible pairs: {(len(sent_ids) *(len(sent_ids)-1))/2:,}')
            
            
            #    logger.info('Sub-sampling..')
            #    sent_id_pairs = random.sample(list(sent_id_pairs), self.sample_n)
            

            for a, b in sent_id_pairs:
                if self.sample_n > 0 and len(texts1) > self.sample_n:
                    logger.info('stop...')
                    break
                    
                self.sent1_ids.append(a)
                self.sent2_ids.append(b)

                texts1.append(self.sent_id2text[a])
                texts2.append(self.sent_id2text[b])

        
        else:
            # Using similarity as candidate filter
            
            logger.info(f'Similarity dataframe size: {len(self.sim_df)}')

            # filter        
            self.sim_df = self.sim_df[(self.sim_df.similarity >= self.min_sim) & (self.sim_df.similarity <= self.max_sim)]

            # sub-sample
            if self.sample_n > 0:
                logger.info('Sub-sampling..')
                self.sim_df = self.sim_df.sample(self.sample_n)

            for a, b in self.sim_df[['sent1_id', 'sent2_id']].values:
                if a in self.sent_id2text and b in self.sent_id2text:
                    self.sent1_ids.append(a)
                    self.sent2_ids.append(b)

                    texts1.append(self.sent_id2text[a])
                    texts2.append(self.sent_id2text[b])
                     
        logger.info(f'Tokenize... {len(texts1):,} samples')

        self.inputs = self.tokenizer(
            text=texts1,
            text_pair=texts2,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors='pt',
            padding='max_length',
            max_length=self.max_length,
            truncation=True,
            return_token_type_ids=True
        )
        
        logger.info('Dataset loaded')

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.inputs.items()}

    def __len__(self):
        return len(self.inputs['input_ids'])

In [14]:
# Load similarity dataframe
sim_df = pd.read_csv(os.path.join(data_dir, 'sentence_pairs-12-02-21.tsv'), sep='\t')
sent_id2text = {sent_id: row['text'] for sent_id, row in meta_df.iterrows()}


In [15]:
sim_df

,sent1_id,sent2_id,similarity
0,146327-5,128379-7,0.941311
1,411276-6,68858-3,0.944875
2,100821-2,100783-28,0.941018
3,170782-2,83662-160,0.946984
4,71884-34,6530-0,0.940196
...,...,...,...
95,83313-15,127214-3,0.940489
96,80368-2,139252-2,0.951340
97,117466-20,189339-7,0.941416
98,53040-2,103883-4,0.940946


In [18]:
# Tokenize dataset
#pred_ds = PredictNSPDataset(sent_id2text, sim_df, tokenizer, min_sim=0, max_sim=1, sample_n=0)
#pred_ds.load()


2021-02-23 15:46:25 - INFO - __main__ -   sent_id2text size: 175720
2021-02-23 15:46:25 - INFO - __main__ -   Similarity dataframe size: 100
2021-02-23 15:46:25 - INFO - __main__ -   Tokenize... 100 samples
2021-02-23 15:46:25 - INFO - __main__ -   Dataset loaded


In [19]:
# Tokenize dataset / without similarity
pred_ds = PredictNSPDataset(sent_id2text, None, tokenizer, sample_n=1_000_000)
pred_ds.load()


2021-02-23 15:46:26 - INFO - __main__ -   sent_id2text size: 175720
2021-02-23 15:46:26 - INFO - __main__ -   Possible pairs: 15,438,671,340.0
2021-02-23 15:46:27 - INFO - __main__ -   stop...
2021-02-23 15:46:27 - INFO - __main__ -   Tokenize... 1,000,001 samples
2021-02-23 15:48:17 - INFO - __main__ -   Dataset loaded


In [20]:
# Load previously trained model from disk
model = BertForNextSentencePrediction.from_pretrained('./output/nsp/bert-base-cased')

In [21]:
pred_trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='./output/nsp__predict/bert-base-cased',
        per_device_eval_batch_size=512,          
    ),
)

In [22]:
pred_out = pred_trainer.predict(pred_ds)

In [23]:
from scipy.special import softmax

nsp_df = pd.DataFrame(dict(
    sent1_id=pred_ds.sent1_ids, 
    sent2_id=pred_ds.sent2_ids, 
    is_next_sentence=softmax(pred_out.predictions[:,0]),
    is_not_next_sentence=softmax(pred_out.predictions[:,1]),
))


nsp_df

,sent1_id,sent2_id,is_next_sentence,is_not_next_sentence
0,736-0,736-1,2.445305e-04,1.028482e-11
1,736-0,741-1,1.119650e-08,1.559122e-06
2,736-0,741-2,1.085477e-08,1.662852e-06
3,736-0,741-3,1.084755e-08,1.551225e-06
4,736-0,741-4,1.123367e-08,1.534388e-06
...,...,...,...,...
999996,741-4,120767-1,9.893848e-09,1.974904e-06
999997,741-4,120767-2,1.027576e-08,1.819374e-06
999998,741-4,120767-3,1.013443e-08,1.883872e-06
999999,741-4,120767-6,9.978086e-09,1.969033e-06


In [27]:
min_score = 0.1

nsp_df[nsp_df.is_next_sentence > nsp_df.is_not_next_sentence]

,sent1_id,sent2_id,is_next_sentence,is_not_next_sentence
0,736-0,736-1,2.445305e-04,1.028482e-11
8,736-0,743-0,6.225773e-05,1.341325e-10
9,736-0,743-1,1.131110e-05,1.016266e-09
12,736-0,797-0,3.154243e-07,7.666976e-08
13,736-0,797-1,3.645239e-05,2.437960e-10
...,...,...,...,...
999945,741-4,120740-5,9.305423e-06,1.425404e-09
999946,741-4,120740-6,2.139979e-07,9.955527e-08
999948,741-4,120740-8,4.410176e-07,4.101011e-08
999950,741-4,120740-10,1.980110e-07,1.126044e-07


In [29]:
nsp_df.to_csv('./output/nsp.1m.csv', index=False)